In [2]:
import numpy as np


import seaborn as sns
import matplotlib.pyplot as plt

import copy, time
import random
import pickle
import scipy

import mlrfit as mf
import hfmfit as hf

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rank = 30
mtype = 'assetcov_gics'

with open("../../mlr_fitting_private/data_finance/crsp_gics_5k_hpart.pkl", "rb") as f:
       gics_hpart = pickle.load(f)

with open('../../mlr_fitting_private/data_finance/crsp_gics_5k_C.npy', 'rb') as f:
       C = np.load(f)

m = n = gics_hpart['rows']['pi'].size
part_sizes = []
for level in range(len(gics_hpart['rows']['lk'])):
    part_sizes += [gics_hpart['rows']['lk'][level].size-1]
    print(f"{level=}, {gics_hpart['rows']['lk'][level].size-1}")

Z = mf.demean_clip(C)
# features are presented in general order
unpermuted_A = Z @ Z.T / (Z.shape[1]-1)

level=0, 1
level=1, 11
level=2, 24
level=3, 69
level=4, 157
level=5, 5000


In [4]:
# permute to put clusters on diagonal
Y = Z[gics_hpart["rows"]["pi"], :].T
N = Y.shape[0]

In [5]:
Y.mean(), Y.max(), Y.min(), np.median(Y)

(-0.00027599359051962765,
 1.7531290483245723,
 -0.6189932233333333,
 -0.0006269800000000001)

In [6]:
num_levels = len(gics_hpart['rows']['lk'])
L = num_levels - 1
num_sparsities = len(gics_hpart['rows']['lk'][L - 1]) - 1
print(f"{num_levels=}, {num_sparsities=}")
# row selector for each sparsity pattern
row_selectors = gics_hpart['rows']['lk'][L - 1]
# traverse hpart tree and assign to each leaf  
# set of group indices 
# from each level to which it belongs
S = []
for level in range(L):
    num_blocks = gics_hpart['rows']['lk'][level].size - 1
    diff = np.diff(gics_hpart['rows']['lk'][level])
    # for each level assign group index to each feature
    S += [np.repeat(np.arange(num_blocks), diff)]
# n x (L-1)
groups_all = np.stack(S, axis=1)

num_levels=6, num_sparsities=157


In [21]:
groups_all[:, ::-1][:, :-1]

array([[  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       ...,
       [156,  68,  23,  10],
       [156,  68,  23,  10],
       [156,  68,  23,  10]])

In [23]:
np.savetxt('asset_cov_Y.dat', Y)

In [24]:
Y.shape, groups_all[:, ::-1][:, :-1].shape

((300, 5000), (5000, 4))

In [27]:
np.savetxt('asset_cov_gics.dat', groups_all[:, ::-1].T)

In [26]:
np.array([14, 6, 4, 3, 2, 1])[::-1]

array([ 1,  2,  3,  4,  6, 14])